# RECOGNIZING TEXT IN IMAGES
## Top Secret Rosies & HP Challenge 2023
### By Anna Gracia Colmenarejo


In [7]:
import pytesseract
import cv2
from pytesseract import Output
import time
import re
import numpy as np
from gtts import gTTS 
from IPython.display import Audio 

In [8]:
def read_text(image):
    start_time = time.time()
    d = pytesseract.image_to_data(image, output_type=Output.DICT)
    time_process = time.time() - start_time
    return d, time_process

In [9]:
def text_detection(d, img):
    
    # Regular expresions to detect patterns
    date_pattern1 = '^(0[1-9]|[12][0-9]|3[01])[- /.](0[1-9]|1[012])[- /.](19|20)\d\d$'
    address_pattern = 'NO[.]'
    date_pattern2 = '\d\d[- /.]\d\d\d\d\d\d\d'
    date_pattern3 = '\d\d\d\d\d[- /.]\d\d\d\d'
    price_pattern = '(\d+\.\d{1,2})'
    
    # Vars
    date = ''
    price = 0.00
    shop_name = ''
    address = ''
    prices = []
    first_word = True

    # print(d.keys())
    n_boxes = len(d['text'])

    for i in range(n_boxes):
        if int(d['conf'][i]) > 60:
            if first_word and re.match('.*', d['text'][i]) and d['text'][i] != " ":
                first_word = False
                aux = i
                (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
                while not re.match("$", d['text'][aux]):
                    w = w + d['width'][aux]
                    shop_name = shop_name + " " + d['text'][aux] 
                    aux += 1
                img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            if re.match(date_pattern1, d['text'][i]):
                (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
                img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                date = d['text'][i]
            if re.findall(date_pattern2, d['text'][i]):
                (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
                img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                if date == '':
                    date = d['text'][i]
            if re.findall(date_pattern3, d['text'][i]):
                (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
                img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                if date == '':
                    date = d['text'][i]
            if re.match(price_pattern, d['text'][i]):
                (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
                img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                prices.append(float(d['text'][i]))
            if re.match(address_pattern, d['text'][i]):
                aux = i
                (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][aux], d['height'][i])
                while not re.match("$", d['text'][aux]):
                    w = w + d['width'][aux] + 3
                    address = address + " " + d['text'][aux] 
                    aux += 1
                img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    try:
        prices = [float(i) for i in prices]
        prices = set(prices)

        if len(prices) >= 2:
            prices.remove(max(prices))

        if len(prices) != 0:
            price = max(prices)

    except:
        price = 0.00

    return shop_name, date, price, address, img


In [10]:
img = cv2.imread('Images/Tickets/test/470.jpg')

pytesseract.pytesseract.tesseract_cmd=r"C:\Program Files\Tesseract-OCR\tesseract"
d, time_process = read_text(img)
shop_name, date, price, address, img = text_detection(d, img)
print("La imatge s'ha processat en %s seconds" % (time_process))
cv2.imshow('img', img)
cv2.waitKey(0)

La imatge s'ha processat en 0.8680009841918945 seconds


-1

In [11]:
text_shop = "The shop's name is {}".format(shop_name)
text_address = "The shop's address is {}".format(address)
text_date = "The purchase date is {}".format(date)
text_price = "The purchase price is {}".format(price)
final_text = text_shop + '\n' + text_address + '\n' + text_date + '\n' + text_price

In [12]:
tts = gTTS(final_text) 
tts.save('detected_text.mp3')
sound_file = 'detected_text.mp3'
Audio(sound_file, autoplay=True) 